In [15]:
import pandas as pd
import numpy as np
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Hardcoded parameters for the specific sweep
entity_name = "dive-ci"
project_name = "CLadder"
# sweep_id = "ngq4k8jt"  # CLadder
sweep_id = "fho761r3"  # ProntoQA

# Initialize wandb API
api = wandb.Api()

# Get the sweep runs
sweep = api.sweep(f"{entity_name}/{project_name}/{sweep_id}")

# dataset_names = {'cladder-v1-q-commonsense': 'commonsense', 'cladder-v1-q-anticommonsense': 'anticommonsense', 'cladder-v1-q-noncommonsense': 'noncommonsense'}
dataset_names = {'prontoqa': 'prontoqa', 'prontoqa-anticommonsense': 'anticommonsense', 'prontoqa-noncommonsense': 'noncommonsense'}

# Collect run data
runs_data = []
for run in sweep.runs:
    # Get run summary and config
    run_data = {
        "id": run.id,
        "name": run.name,
        "state": run.state,
        "created_at": run.created_at,
        "summary": run.summary._json_dict if hasattr(run.summary, "_json_dict") else {},
        "config": run.config
    }
    
    # Extract relevant metrics and parameters
    summary = run_data["summary"]
    config = run_data["config"]
    # print(summary)
    
    run_info = {
        "run_id": run.id,
        "run_name": run.name,
        "reasoning": config.get("experiment.reasoning", None),
        "anonymize": config.get("dataset.anonymize", None),
        "learning_rate": config.get("training.learning_rate", None),
        "percent_train": config.get("dataset.percent_of_train_dataset", None),
        # "num_epochs": config.get("training.num_train_epochs", None),
        "max_steps": config.get("training.max_steps", None),
    }
    
    # Extract scores for different test datasets
    for data_name, score_key in dataset_names.items():
        try: 
            run_info[score_key] = summary[data_name]
        except KeyError:
            print(f"KeyError: {data_name} not found in summary for run {summary}")
    
    runs_data.append(run_info)

# Convert to dataframe
df = pd.DataFrame(runs_data)

# Identify test dataset columns
test_dataset_columns = [col for col in df.columns if col in dataset_names.values()]
training_percentages = sorted(df['percent_train'].unique())
learning_rate = sorted(df['learning_rate'].unique())
anonymize_list = sorted(df['anonymize'].unique())

print(f"Found {len(df)} runs with {len(test_dataset_columns)} test datasets")
print(f"Test datasets: {[col.replace('score_', '') for col in test_dataset_columns]}")
print(f"Training percentages: {training_percentages}")
df

KeyError: prontoqa not found in summary for run {'_runtime': 129.40086960792542, '_step': 2, '_timestamp': 1744756456.2190456, 'total_flos': 899428726210560, 'train/epoch': 1, 'train/global_step': 50, 'train/grad_norm': 0.3702305853366852, 'train/learning_rate': 0, 'train/loss': 0.2162, 'train/mean_token_accuracy': 0.929225664138794, 'train_loss': 0.4447970390319824, 'train_runtime': 130.407, 'train_samples_per_second': 1.534, 'train_steps_per_second': 0.383}
KeyError: prontoqa-anticommonsense not found in summary for run {'_runtime': 129.40086960792542, '_step': 2, '_timestamp': 1744756456.2190456, 'total_flos': 899428726210560, 'train/epoch': 1, 'train/global_step': 50, 'train/grad_norm': 0.3702305853366852, 'train/learning_rate': 0, 'train/loss': 0.2162, 'train/mean_token_accuracy': 0.929225664138794, 'train_loss': 0.4447970390319824, 'train_runtime': 130.407, 'train_samples_per_second': 1.534, 'train_steps_per_second': 0.383}
KeyError: prontoqa-noncommonsense not found in summary f

,run_id,run_name,reasoning,anonymize,learning_rate,percent_train,max_steps,prontoqa,anticommonsense,noncommonsense
0,zbq0r8g4,QWen-prontoqa-RTrue-DP0.1-s3200-Absrandom-lr0....,True,random,0.00030,0.100,3200,96.75,89.00,94.75
1,bm4thbr4,QWen-prontoqa-RTrue-DP0.1-s1600-Absrandom-lr0....,True,random,0.00030,0.100,1600,96.50,87.00,93.50
2,c17xgyyz,QWen-prontoqa-RTrue-DP0.1-s800-Absrandom-lr0.0003,True,random,0.00030,0.100,800,95.75,87.25,92.00
3,27yvyamr,QWen-prontoqa-RTrue-DP0.1-s400-Absrandom-lr0.0003,True,random,0.00030,0.100,400,93.75,85.00,91.25
4,g3jjbwuu,QWen-prontoqa-RTrue-DP0.1-s200-Absrandom-lr0.0003,True,random,0.00030,0.100,200,87.50,81.50,85.00
...,...,...,...,...,...,...,...,...,...,...
135,d236lzux,QWen-prontoqa-RTrue-DP0.005-s400-AbsNone-lr0.0...,True,null,0.00015,0.005,400,96.00,65.75,76.50
136,i4tn46z0,QWen-prontoqa-RTrue-DP0.005-s50-AbsNone-lr0.00015,True,null,0.00015,0.005,50,86.50,65.00,77.50
137,iwkhq5ke,QWen-prontoqa-RTrue-DP0.005-s1600-AbsNone-lr0....,True,null,0.00015,0.005,1600,93.25,65.00,72.75
138,r9mp25v7,QWen-prontoqa-RTrue-DP0.005-s100-AbsNone-lr0.0...,True,null,0.00015,0.005,100,87.50,58.25,75.00


In [16]:
sweep = api.sweep(f"{entity_name}/{project_name}/{sweep_id}")
sweep.runs[0].config["dataset.percent_of_train_dataset"]
# for key in sweep.runs[1].__dir__():
#     print(sweep.runs[0].getattr(key))



0.1

In [19]:

# Create comparison plots
def create_anonymize_comparison_plots(learning_rate):
    # Create a subplot for each test dataset and training percentage combination
    num_test_sets = len(test_dataset_columns)
    num_percentages = len(training_percentages)
    
    fig = make_subplots(
        rows=num_percentages, 
        cols=num_test_sets,
        subplot_titles=[f"{col} - {pct*100}% Training" 
                        for pct in training_percentages for col in test_dataset_columns],
        vertical_spacing=0.05,
        horizontal_spacing=0.05
    )
    
    # Color scheme for True/False anonymization
    colors = {"null": "rgb(31, 119, 180)", "random": "rgb(255, 127, 14)"}
    
    for i, pct in enumerate(training_percentages):
        for j, test_col in enumerate(test_dataset_columns):
            dataset_name = test_col
            
            # Filter data for this training percentage
            df_pct = df[(df['percent_train'] == pct) & (df['learning_rate'] == learning_rate)]
            
            # Group by anonymization and epochs, calculate mean scores
            for anon in anonymize_list:
                df_group = df_pct[df_pct['anonymize'] == anon]
                
                # Skip if no data for this combination
                if len(df_group) == 0:
                    continue
                
                # Get average scores for each epoch
                # print(df_group)
                epoch_scores = df_group.groupby('max_steps')[test_col].mean().reset_index()
                # print(epoch_scores)
                # print(epoch_scores)
                
                # Add line to plot
                fig.add_trace(
                    go.Scatter(
                        x=epoch_scores['max_steps'],
                        y=epoch_scores[test_col],
                        mode='lines+markers',
                        name=f"Anonymize={anon}",
                        line=dict(color=colors[str(anon)]),
                        legendgroup=f"Anonymize={anon}",
                        showlegend=(i==0 and j==0) # Only show legend once
                    ),
                    row=i+1, col=j+1
                )
            
            # Update axis labels
            if i == num_percentages-1:
                fig.update_xaxes(title_text="Number of Epochs", row=i+1, col=j+1)
            if j == 0:
                fig.update_yaxes(title_text="Score", row=i+1, col=j+1)
    
    # Update layout
    fig.update_layout(
        height=200*num_percentages,
        width=400*num_test_sets,
        title_text="Anonymize vs Non-Anonymize Performance Comparison",
        legend_title="Anonymization Setting",
        margin=dict(t=50, b=20, l=20, r=20),
    )
    
    return fig

# Create and display the comparison plots
for lr in learning_rate:
    print(f"Creating comparison plots for learning rate: {lr}")
    comparison_fig = create_anonymize_comparison_plots(lr)
    comparison_fig.show()

Creating comparison plots for learning rate: 0.00015


Creating comparison plots for learning rate: 0.0003


In [8]:

# Create a simplified single-row comparison for easier viewing
def create_simplified_comparison():
    # Create one row of plots, one for each test dataset
    fig = make_subplots(
        rows=1, 
        cols=len(test_dataset_columns),
        subplot_titles=[col.replace('score_', '') for col in test_dataset_columns],
        horizontal_spacing=0.1
    )
    
    # Color scheme for different training percentages
    colors = px.colors.qualitative.Plotly
    
    for j, test_col in enumerate(test_dataset_columns):
        dataset_name = test_col.replace('score_', '')
        
        # For each training percentage
        for i, pct in enumerate(training_percentages):
            # Filter data for this training percentage
            df_pct = df[df['percent_train'] == pct]
            
            # For anonymize=True and anonymize=False
            for k, anon in enumerate([True, False]):
                df_group = df_pct[df_pct['anonymize'] == anon]
                
                # Skip if no data for this combination
                if len(df_group) == 0:
                    continue
                
                # Get average scores for each epoch
                epoch_scores = df_group.groupby('num_epochs')[test_col].mean().reset_index()
                
                # Add line to plot
                fig.add_trace(
                    go.Scatter(
                        x=epoch_scores['num_epochs'],
                        y=epoch_scores[test_col],
                        mode='lines+markers',
                        name=f"{pct*100}% Train, Anon={anon}",
                        line=dict(
                            color=colors[i % len(colors)],
                            dash='dash' if anon else 'solid'
                        ),
                    ),
                    row=1, col=j+1
                )
            
            # Update axis labels
            fig.update_xaxes(title_text="Number of Epochs", row=1, col=j+1)
            if j == 0:
                fig.update_yaxes(title_text="Score", row=1, col=j+1)
    
    # Update layout
    fig.update_layout(
        height=500,
        width=350*len(test_dataset_columns),
        title_text="Anonymize vs Non-Anonymize Performance Comparison",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5
        ),
        margin=dict(b=100)
    )
    
    return fig

# Create and display the simplified comparison
simple_fig = create_simplified_comparison()
simple_fig.show()